## CSS Project Part 2
Arathy Pillai, Alexander Guth, Hammad Akthar, Hiba Saleel and Julian Jetz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation

In [2]:
plot_size = plt.rcParams["figure.figsize"]  
plot_size[0] = 15  
plot_size[1] = 8  
plt.rcParams["figure.figsize"] = plot_size  

### Load data and determine na values
repost and Text are determined as na values, because they ... **[TODO]**

In [3]:
na_values = ['nan', 'N/A', 'NaN', 'NaT', '', 'Text', 'repost', 'r e p o s t -', 'Repost']
#na_values = ['nan', 'N/A', 'NaN', 'NaT', '0', '', 'Text', 'repost', 'r e p o s t -', 'Repost']
ira_data = pd.read_csv('../Data/data_IRA_Ads.csv', sep=";", parse_dates=['AD_CREATION_DATE', 'AD_END_DATE'], 
                       dayfirst=False, index_col='AD_ID', na_values=na_values)
ira_data = ira_data[ira_data['AD_IMPRESSIONS']>0]

### Click per View Ratio
#Clicks / #Impressions

In [4]:
ira_data.insert(5, 'AD_CLICKS_PER_VIEW', ira_data['AD_CLICKS']/ira_data['AD_IMPRESSIONS'])

In [21]:
ira_data['AD_CREATION_WEEKDAY'] = ira_data['AD_CREATION_DATE'].dt.day_name()
ira_data['AD_CREATION_YEAR'] = ira_data['AD_CREATION_DATE'].dt.year
ira_data['AD_CREATION_MONTH'] = ira_data['AD_CREATION_DATE'].dt.month
ira_data['AD_CREATION_DAY'] = ira_data['AD_CREATION_DATE'].dt.day
ira_data['AD_CREATION_TIME'] = ira_data['AD_CREATION_DATE'].dt.time
ira_data['AD_CREATION_TIMESTAMP'] = ira_data['AD_CREATION_DATE'].dt.timestamp()

AttributeError: 'DatetimeProperties' object has no attribute 'timestamp'

### Convert ruble
Removes "RUB" from the cell AD_SPEND

In [6]:
ira_data['AD_SPEND']=ira_data['AD_SPEND'].str.replace(' RUB','')

### Drop all subsets, where AD_TEXT is na

In [7]:
ira_data=ira_data.dropna(subset=['AD_TEXT']);
ira_data['AD_TEXT'].count()

2533

### Remove hyperlinks

In [8]:
ira_data['AD_TEXT']=ira_data['AD_TEXT'].str.replace(r'https?:\/\/.*[\r\n]*', '')
ira_data['AD_TEXT']=ira_data['AD_TEXT'].str.replace(r'http?:\/\/.*[\r\n]*', '')

### Select all duplicate ad texts

In [9]:
duplicateDF = pd.concat(g for _, g in ira_data.groupby("AD_TEXT") if len(g) > 1).copy()
duplicateDF.sort_values(by=['AD_TEXT'], inplace=True)

In [10]:
ira_data['AD_TEXT'].count()

2533

In [11]:
uniqueDF = ira_data.copy()
uniqueDF.drop_duplicates(subset ='AD_TEXT', 
                     keep = False, inplace = True)

### Total Duplicates

In [12]:
ira_data['AD_TEXT'].count()

2533

In [13]:
duplicateDF['AD_TEXT'].count()

614

In [14]:
uniqueDF['AD_TEXT'].count()

1919

### Unique Duplicate Pairs

In [15]:
duplicateDF['AD_TEXT'].nunique()

203

### Occurence of Duplicate Pairs

In [16]:
duplicateDF['AD_TEXT'].value_counts().values

array([13, 12,  9,  8,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

In [17]:
duplicateDF[duplicateDF['AD_IMPRESSIONS']>0]['AD_TEXT'].value_counts()

We speak for all fellow members of LGBT community across the nation.                                                                                                                                                                                                                                                      13
We are proud to be black and stand for our community! Join to end the racism in the US.                                                                                                                                                                                                                                   12
Welcome to the city of memes - Memopolis.                                                                                                                                                                                                                                                                                  9
First online player for ur browser! Listen everyt

In [18]:
duplicateDF[(duplicateDF['AD_IMPRESSIONS']>0) & duplicateDF['AD_TEXT'].str.match('We speak for all fellow members of LGBT community across the nation.')][['AD_CLICKS', 'AD_IMPRESSIONS', 'AD_CREATION_DATE', 'AD_CREATION_WEEKDAY','AD_CREATION_YEAR', 'AD_CREATION_MONTH','AD_CREATION_DAY','AD_CREATION_TIME','AD_TEXT', 'AD_SPEND', 'AD_POTENTIAL_TARGET']].head(20)

,AD_CLICKS,AD_IMPRESSIONS,AD_CREATION_DATE,AD_CREATION_WEEKDAY,AD_CREATION_YEAR,AD_CREATION_MONTH,AD_CREATION_DAY,AD_CREATION_TIME,AD_TEXT,AD_SPEND,AD_POTENTIAL_TARGET
AD_ID,,,,,,,,,,,
669,91,1997,2015-06-17 03:15:18,Wednesday,2015,6,17,03:15:18,We speak for all fellow members of LGBT commun...,2673.94,144000000.0
591,5441,93925,2016-03-23 05:36:11,Wednesday,2016,3,23,05:36:11,We speak for all fellow members of LGBT commun...,31713.12,NaN
590,6566,67221,2015-11-05 23:11:58,Thursday,2015,11,5,23:11:58,We speak for all fellow members of LGBT commun...,46994.76,36000000.0
588,21449,175460,2015-07-14 00:01:38,Tuesday,2015,7,14,00:01:38,We speak for all fellow members of LGBT commun...,93754.38,41000000.0
587,542,15175,2015-07-09 01:50:28,Thursday,2015,7,9,01:50:28,We speak for all fellow members of LGBT commun...,6201.70,35000000.0
586,59,1811,2015-07-07 01:51:07,Tuesday,2015,7,7,01:51:07,We speak for all fellow members of LGBT commun...,757.65,166000000.0
592,7289,190076,2016-08-04 04:20:05,Thursday,2016,8,4,04:20:05,We speak for all fellow members of LGBT commun...,36000.00,NaN
584,3206,31618,2015-06-23 05:06:57,Tuesday,2015,6,23,05:06:57,We speak for all fellow members of LGBT commun...,16630.20,143000000.0
583,186,4265,2015-06-22 08:19:07,Monday,2015,6,22,08:19:07,We speak for all fellow members of LGBT commun...,1393.55,139000000.0


In [19]:
duplicateDF.sample(25)

,AD_CLICKS,AD_CREATION_DATE,AD_END_DATE,AD_IMPRESSIONS,AD_LANDING_PAGE,AD_CLICKS_PER_VIEW,AD_PATH,AD_SPEND,AD_TARGETING_AGE,AD_TARGETING_CUSTOM_AUDIENCE,...,AD_TARGETING_LOCATION_RECENTLY_IN,AD_TARGETING_PEOPLE_WHO_MATCH,AD_TARGETING_PLACEMENTS,AD_TEXT,AD_POTENTIAL_TARGET,AD_CREATION_WEEKDAY,AD_CREATION_YEAR,AD_CREATION_MONTH,AD_CREATION_DAY,AD_CREATION_TIME
AD_ID,,,,,,,,,,,,,,,,,,,,,
2775,2927,2017-01-16 03:06:47,NaT,41280,https://www.facebook.com/south-united-17770373...,0.070906,2017-q1/2017-01/P(1)0005310.pdf,9370.63,18 - 65+,NaN,...,NaN,NaN,"News Feed on desktop computers, News Feed on m...","Heritage, not hate. The South will rise again!...",220000.0,Monday,2017,1,16,03:06:47
3480,0,2016-05-11 01:43:47,NaT,17,https://musicfb.info/,0.000000,2016-q2/2016-05/P(1)0006129.pdf,NaN,13 - 30,NaN,...,NaN,Behaviors: Facebook access (browser): Chrome A...,News Feed on desktop computers,Free online player! Jump in the world of free ...,NaN,Wednesday,2016,5,11,01:43:47
1624,76,2016-06-29 02:00:21,2016-07-06 03:27:59,1293,https://www.facebook.com/events/1170988439609888/,0.058778,2016-q2/2016-06/P(1)0003673.pdf,1199.85,16 - 53,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,The time has come to understand one simple thi...,NaN,Wednesday,2016,6,29,02:00:21
3392,5,2017-03-10 01:29:58,NaT,147,https://www.facebook.com/eterans-come-first-i7...,0.034014,2017-q1/2017-03/P(1)0005951.pdf,147.50,18 - 65+,NaN,...,NaN,NaN,"News Feed on desktop computers, News Feed on m...",Support our Veterans! Veterans Come First,141000000.0,Friday,2017,3,10,01:29:58
966,3793,2017-03-23 02:44:02,2017-03-24 11:00:00,22589,https://www.facebook.com/blacktivists/,0.167914,2017-q1/2017-03/P(1)0002805.pdf,200.00,16 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Alek Wek is now a hero for many of us. Supermo...,24000000.0,Thursday,2017,3,23,02:44:02
3347,74,2017-03-31 06:57:14,2017-04-01 06:57:14,1974,https://www.meetup.com/black-fist-self-defense...,0.037487,2017-q1/2017-03/P(1)0006586.pdf,1000.00,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,"Join the event, bring your friends, feel safe ...",NaN,Friday,2017,3,31,06:57:14
1454,1,2016-04-13 07:33:11,NaT,77,https://www.facebook.com/events/1748220542079708/,0.012987,2016-q2/2016-04/P(1)0003476.pdf,55.74,19 - 30,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Every voice counts! Join us and stop police br...,NaN,Wednesday,2016,4,13,07:33:11
2652,2151,2017-04-13 07:48:53,2017-04-14 07:48:53,15285,https://www.facebook.com/brownunitedfront/,0.140726,2017-04/P(1)0005163.pdf,276.53,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,So true!,43000000.0,Thursday,2017,4,13,07:48:53
796,1,2016-01-14 06:33:13,NaT,46,https://www.facebook.com/viiiiamsandkaivin/,0.021739,2016-q1/2016-01/P(1)0001365.pdf,114.87,18 - 45,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,It's time for changes in Black Community! Stay...,7700000.0,Thursday,2016,1,14,06:33:13


In [20]:
uniqueDF.sample(25)

,AD_CLICKS,AD_CREATION_DATE,AD_END_DATE,AD_IMPRESSIONS,AD_LANDING_PAGE,AD_CLICKS_PER_VIEW,AD_PATH,AD_SPEND,AD_TARGETING_AGE,AD_TARGETING_CUSTOM_AUDIENCE,...,AD_TARGETING_LOCATION_RECENTLY_IN,AD_TARGETING_PEOPLE_WHO_MATCH,AD_TARGETING_PLACEMENTS,AD_TEXT,AD_POTENTIAL_TARGET,AD_CREATION_WEEKDAY,AD_CREATION_YEAR,AD_CREATION_MONTH,AD_CREATION_DAY,AD_CREATION_TIME
AD_ID,,,,,,,,,,,,,,,,,,,,,
516,222,2015-09-17 07:28:27,2015-09-18 07:28:26,1318,https://www.facebook.com/patriototus/,0.168437,2015-q3/2015-09/P(1)0002375.pdf,351.16,18 - 65+,NaN,...,NaN,"People who like Being Patriotic,",News Feed on desktop computers or News Feed on...,The U.S. Constitutional Convention signed and ...,NaN,Thursday,2015,9,17,07:28:27
2805,18,2016-11-15 05:19:09,2016-11-16 05:19:09,125,https://www.facebook.com/nefertitis-community-...,0.144000,2016-q4/2016-11/P(1)0005356.pdf,39.76,18 - 60,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,All power to the black community! #black #fist...,21000000.0,Tuesday,2016,11,15,05:19:09
93,922,2015-11-19 23:59:53,NaT,17231,https://www.facebook.com/events/1486230091674577/,0.053508,2015-q4/2015-11/P(1)0001870.pdf,3988.68,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,A Year Without Justice for Tamir Rice. Join th...,NaN,Thursday,2015,11,19,23:59:53
288,345,2016-07-22 02:39:14,2016-07-23 02:39:12,3203,https://www.facebook.com/black-matters-1579673...,0.107712,2016-q3/2016-07/P(1)0000053.pdf,500.00,18 - 54,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Individual protesters in Oakland refused to at...,31000000.0,Friday,2016,7,22,02:39:14
2508,171,2016-11-11 04:57:56,NaT,2159,https://represent.com/not-my-president-1,0.079203,2016-q4/2016-11/P(1)0004965.pdf,584.08,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Are you unhappy with our new president his big...,25000000.0,Friday,2016,11,11,04:57:56
3251,1342,2017-04-25 05:56:10,NaT,28502,https://www.facebook.com/brownunitedfront/,0.047084,2017-04/P(1)0005865.pdf,4413.97,18 - 65+,NaN,...,NaN,NaN,"News Feed on desktop computers, News Feed on m...","Brown Power is a platform designed to educate,...",580000.0,Tuesday,2017,4,25,05:56:10
295,782,2016-08-10 00:32:13,2016-08-11 00:32:11,8822,https://www.facebook.com/blackmattersus//,0.088642,2016-q3/2016-08/P(1)0002067.pdf,866.98,18 - 65+,"Lookalike(US, 1%)- newtestaudit",...,NaN,NaN,News Feed on desktop computers or News Feed on...,MOVE heroes need your support! Those who fough...,NaN,Wednesday,2016,8,10,00:32:13
1981,1566,2017-04-28 06:55:38,2017-04-29 06:55:38,6942,https://www.facebook.com/blackmattersus/.mvmnt/,0.225583,2017-04/P(1)0004147.pdf,263.59,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Miss this show. Like and share.,2900000.0,Friday,2017,4,28,06:55:38
595,209,2015-07-17 06:34:31,2015-07-19 06:34:31,1617,https://www.facebook.com/lgbt-united-839497472...,0.129252,2015-q3/2015-07/P(1)0002496.pdf,249.98,18 - 65+,NaN,...,NaN,People who like LGBT United.,News Feed on desktop computers or News Feed on...,HOMOPHOBIA STILL RUNNING STRONG IN AMERICA. Co...,NaN,Friday,2015,7,17,06:34:31
